**Pontifícia Universidade Católica do Rio de Janeiro – PUC-RJ**

Disciplina de Engenharia de Dados 

Aluno: Marcos Gabriel Fernandes Andrade

In [0]:
df = spark.read.csv("/FileStore/MVP/suicidios.csv", sep=";", header=True, inferSchema=True)

Importando o arquivo CSV do caminho: FilesSore/MVP/suicidios.csv

In [0]:
new_columns = [
    "estado", "ano", "circobito", "dtobito", "dtnasc", "sexo", "racacor", 
    "estciv", "esc", "ocup", "codmunres", "lococor", "assistmed", 
    "causabas", "causabas_o", "idade", "mes"
]
df = df.toDF(*new_columns)

Colunas renomeadas para garantir que não haja caracteres inválidos

In [0]:
df.write.saveAsTable("suicidios")

---------------------------------------------------------------------------
AnalysisException                         Traceback (most recent call last)
File <command-1660055847274372>:1
----> 1 df.write.saveAsTable("suicidios")

File /databricks/spark/python/pyspark/instrumentation_utils.py:48, in _wrap_function.<locals>.wrapper(*args, **kwargs)
     46 start = time.perf_counter()
     47 try:
---> 48     res = func(*args, **kwargs)
     49     logger.log_success(
     50         module_name, class_name, function_name, time.perf_counter() - start, signature
     51     )
     52     return res

File /databricks/spark/python/pyspark/sql/readwriter.py:1520, in DataFrameWriter.saveAsTable(self, name, format, mode, partitionBy, **options)
   1518 if format is not None:
   1519     self.format(format)
-> 1520 self._jwrite.saveAsTable(name)

File /databricks/spark/python/lib/py4j-0.10.9.5-src.zip/py4j/java_gateway.py:1321, in JavaMember.__call__(self, *args)
   1315 command = proto.CALL_COMM

Erro na execução do comando df.write, haja vista que o diretório já possuia dados armazenados

In [0]:
dbutils.fs.rm("dbfs:/user/hive/warehouse/suicidios", recurse=True)

Out[5]: True

usado o comando dbutils.fs.rm para limpar o diretório

In [0]:
df.write.saveAsTable("suicidios")

Consulta em SQL para confirmar se a tabela criada foi criada com êxito

In [0]:
%sql
select *
from suicidios
limit 10

estado,ano,circobito,dtobito,dtnasc,sexo,racacor,estciv,esc,ocup,codmunres,lococor,assistmed,causabas,causabas_o,idade,mes
AC,2014,Suicidio,2014-01-02,1977-07-02,Masculino,Preta,Casado,4 a 7 anos,ATLETA PROFISSIONAL DE FUTEBOL,Rio Branco,Domicilio,Nao,X700,X700,37,1
AC,2014,Suicidio,2014-01-23,1966-07-30,Masculino,Branca,Uniao Consensual,12 e mais,MEDICO GINECOLOGISTA E OBSTETRA,Rio Branco,Domicilio,Nao,X800,X800,48,1
AC,2014,Suicidio,2014-01-31,1943-07-28,Masculino,Branca,NA,NA,0,Rio Branco,Domicilio,Nao,X700,X700,71,1
AC,2014,Suicidio,2014-02-05,1999-07-04,Masculino,Branca,Solteiro,4 a 7 anos,ESTUDANTE,Epitaciol�ndia,Outros,Nao,X700,X700,15,2
AC,2014,Suicidio,2014-02-06,1972-05-02,Masculino,Branca,NA,NA,0,Rio Branco,Outros,Nao,X700,X700,42,2
AC,2014,Suicidio,2014-02-21,1963-06-10,Masculino,Parda,Solteiro,8 a 11 anos,0,Rio Branco,Domicilio,Nao,X700,X700,51,2
AC,2014,Suicidio,2014-02-27,1989-08-06,Masculino,Parda,Solteiro,1 a 3 anos,0,Senador Guiomard,Outros,Nao,X700,X700,25,2
AC,2014,Suicidio,2014-03-04,1961-06-16,Feminino,Branca,Casado,NA,EMPREGADO DOMESTICO DIARISTA,Rio Branco,Domicilio,Nao,X700,X700,53,3
AC,2014,Suicidio,2014-03-10,1975-02-28,Masculino,Branca,Casado,12 e mais,0,Cruzeiro do Sul,Domicilio,Nao,X702,X702,39,3
AC,2014,Suicidio,2014-03-24,1977-10-09,Feminino,Parda,Solteiro,8 a 11 anos,DONA DE CASA,Feij�,Domicilio,null,X749,X749,37,3


Transformação e qualidade dos dados:
Verificar se existem valores difentes de "Suicidio" na coluna circunstância do óbito "circobito". Este processo se faz necessário para garantir que todas as circunstâncias dos óbitos sejam o objeto da análise deste MVP.

In [0]:
%sql
SELECT *
FROM suicidios
WHERE circobito != 'Suicidio'
limit 10

estado,ano,circobito,dtobito,dtnasc,sexo,racacor,estciv,esc,ocup,codmunres,lococor,assistmed,causabas,causabas_o,idade,mes
AC,2014,Acidente,2014-08-31,1976-08-16,Masculino,Parda,Uniao Consensual,Nenhuma,DONA DE CASA,Feij�,Domicilio,Nao,X700,X700,38,8
AC,2014,Acidente,2014-11-20,1962-08-03,Feminino,Parda,Casado,Nenhuma,DONA DE CASA,Feij�,Domicilio,null,X700,X700,52,11
AL,2014,NA,2014-02-11,1939-09-24,Feminino,Parda,Viuvo,Nenhuma,APOSENTADO/PENSIONISTA,Arapiraca,Domicilio,Nao,X699,X699,75,2
AL,2014,NA,2014-02-28,1983-04-28,Feminino,Parda,Casado,8 a 11 anos,EMPREGADO DOMESTICO NOS SERVICOS GERAIS,Coit� do N�ia,Domicilio,null,X700,X700,31,2
AL,2014,Acidente,2014-04-15,1973-11-04,Masculino,Parda,Solteiro,NA,0,Macei�,Domicilio,null,X709,X709,41,4
AL,2014,NA,2014-06-09,1969-06-13,Masculino,Parda,Solteiro,NA,TRABALHADOR VOLANTE DA AGRICULTURA,Estrela de Alagoas,Domicilio,Nao,X709,X709,45,6
AM,2014,NA,2014-01-16,1993-12-13,Masculino,Indigena,Solteiro,8 a 11 anos,0,Tabatinga,Domicilio,NA,X700,X700,21,1
AM,2014,NA,2014-01-25,1942-12-20,Masculino,Parda,Viuvo,4 a 7 anos,0,Codaj�s,Domicilio,null,X700,X700,72,1
AM,2014,Acidente,2014-01-31,1973-09-08,Feminino,Parda,Solteiro,8 a 11 anos,AGENTE DE SAUDE PUBLICA,Manaus,Domicilio,null,X700,X700,41,1
AM,2014,NA,2014-02-01,1997-06-02,Masculino,Indigena,Solteiro,1 a 3 anos,PRODUTOR AGRICOLA POLIVALENTE,Atalaia do Norte,Outro estabelecimento de saude,Nao,X789,X789,17,2


Com a consulta realizada, foi verificado que na coluna "circobito" existem valores diferentes de "Suicidio". Realizado o comando DELELE da linguagem SQL para excluir esses valores e também valores nulos, se houver. 

In [0]:
%sql
DELETE FROM suicidios
WHERE circobito != 'Suicidio' OR circobito IS NULL

num_affected_rows
3150


Excluído 3150 linhas da base de dados. Nos póximos comandos, foi realizado uma consulta para verificar se existem valores nulos nas colunas "estado" e "ano".

In [0]:
%sql
SELECT *
FROM suicidios
WHERE estado is null 

estado,ano,circobito,dtobito,dtnasc,sexo,racacor,estciv,esc,ocup,codmunres,lococor,assistmed,causabas,causabas_o,idade,mes


In [0]:
%sql
SELECT *
FROM suicidios
WHERE ano is null 

estado,ano,circobito,dtobito,dtnasc,sexo,racacor,estciv,esc,ocup,codmunres,lococor,assistmed,causabas,causabas_o,idade,mes


Verificado que não existem valores nulos nas colunas 'estado' e 'ano'. 

O comando a seguir verificou se a coluna 'ano' apresentava valores fora do intervalo da pesquisa (2014 a 2018). A análise confirmou que todos os registros estão dentro do período definido.

In [0]:
%sql
SELECT DISTINCT ano
FROM suicidios
ORDER BY ano

ano
2014
2015
2016
2017
2018


Transformação dos dados: Foi verificado que a coluna com a causa básica do óbito 'causabas' estava duplicada, com isso, foi realizado o comando DROP COLUMN para excluir da base a coluna duplicada 'causabas_o'.

In [0]:
%sql
ALTER TABLE spark_catalog.default.suicidios DROP COLUMN causabas_o

---------------------------------------------------------------------------
AnalysisException                         Traceback (most recent call last)
File <command-562308212959090>:7
      5     display(df)
      6     return df
----> 7   _sqldf = ____databricks_percent_sql()
      8 finally:
      9   del ____databricks_percent_sql

File <command-562308212959090>:4, in ____databricks_percent_sql()
      2 def ____databricks_percent_sql():
      3   import base64
----> 4   df = spark.sql(base64.standard_b64decode("QUxURVIgVEFCTEUgc3VpY2lkaW9zIApEUk9QIENPTFVNTiBjYXVzYWJhc19v").decode())
      5   display(df)
      6   return df

File /databricks/spark/python/pyspark/instrumentation_utils.py:48, in _wrap_function.<locals>.wrapper(*args, **kwargs)
     46 start = time.perf_counter()
     47 try:
---> 48     res = func(*args, **kwargs)
     49     logger.log_success(
     50         module_name, class_name, function_name, time.perf_counter() - start, signature
     51     )
     52     r

O comando DROP COLUMN não está disponível nesta versão do Databricks. Com isso, foi gerado um comando na linguagem SQL para criar uma nova tabela sem a coluna duplicada.

In [0]:
%sql
CREATE OR REPLACE TABLE spark_catalog.default.suicidios_temp AS
SELECT 
  estado, ano, circobito, dtobito, dtnasc, sexo, racacor, estciv, esc, ocup, codmunres, lococor, assistmed, causabas, idade, mes
FROM spark_catalog.default.suicidios


num_affected_rows,num_inserted_rows


In [0]:
%sql
DROP TABLE spark_catalog.default.suicidios

In [0]:
%sql
ALTER TABLE spark_catalog.default.suicidios_temp 
RENAME TO spark_catalog.default.suicidios;

---------------------------------------------------------------------------
AnalysisException                         Traceback (most recent call last)
File <command-562308212959095>:7
      5     display(df)
      6     return df
----> 7   _sqldf = ____databricks_percent_sql()
      8 finally:
      9   del ____databricks_percent_sql

File <command-562308212959095>:4, in ____databricks_percent_sql()
      2 def ____databricks_percent_sql():
      3   import base64
----> 4   df = spark.sql(base64.standard_b64decode("QUxURVIgVEFCTEUgc3BhcmtfY2F0YWxvZy5kZWZhdWx0LnN1aWNpZGlvc190ZW1wIApSRU5BTUUgVE8gc3BhcmtfY2F0YWxvZy5kZWZhdWx0LnN1aWNpZGlvcw==").decode())
      5   display(df)
      6   return df

File /databricks/spark/python/pyspark/instrumentation_utils.py:48, in _wrap_function.<locals>.wrapper(*args, **kwargs)
     46 start = time.perf_counter()
     47 try:
---> 48     res = func(*args, **kwargs)
     49     logger.log_success(
     50         module_name, class_name, function_name, ti

Após a criação da nova planilha sem a coluna duplicada, o arquivo antigo foi deletado e executado o comando RENAME da linguagem SQL para renomear a nova tabela, entretanto, o comando apresentou o erro acima, visto que a tabela está armazenada no S3, e o databricks por padrão bloqueia essa operação. Como solução, foi utilizado o comando RENAME da linguagem spark.

In [0]:

spark.conf.set("spark.databricks.delta.alterTable.rename.enabledOnAWS", "true")


spark.sql("""
ALTER TABLE spark_catalog.default.suicidios_temp 
RENAME TO spark_catalog.default.suicidios
""")


spark.conf.set("spark.databricks.delta.alterTable.rename.enabledOnAWS", "false")

Após realizado a etapa de transformação e verificado a qualidade dos dados, foi iniciado os scripts para respostas das perguntas definidas no objetivo deste MVP.

Pergunta 01: Analisar o número de suicídios por ano.
Esta pergunta visa avaliar se os óbitos por suicídio estão crescendo a cada ano, conforme o período da análise.


In [0]:
%sql
SELECT ano, COUNT(*) AS total_obitos
FROM suicidios
GROUP BY ano
ORDER BY ano

ano,total_obitos
2014,10220
2015,10531
2016,10683
2017,11798
2018,12252


Conforme o resultado da consulta, os suicídios no Brasil aumentaram 16% entre 2014 e 2018. Esse crescimento coincide com o avanço de diagnósticos de depressão e ansiedade, segundo a PNS (IBGE, 2019). A OMS ainda destaca que transtornos mentais são fatores de risco em 90% dos casos de suicídio, reforçando a possível correlação entre os dados.

Pergunta 02: Analisar o número de suicídios por estado

In [0]:
%sql
SELECT estado, COUNT(*) AS total_obitos
FROM suicidios
GROUP BY estado
ORDER BY total_obitos DESC

estado,total_obitos
SP,10221
MG,6658
RS,5983
PR,3655
SC,3233
CE,2876
RJ,2859
BA,2508
GO,2312
PE,1749


São Paulo registrou o maior númerode suicídios no período analisado, no entanto, esse dado deve ser analisado considerando que o estado é o mais populoso do país, com aproximadamente 44 milhões de habitantes em 2018 (IBGE), o que representa cerca de 22% da população brasileira. Quando ajustamos os valores à densidade populacional, observamos que outras unidades federativas como Minas Gerais (21 milhões), Rio Grande do Sul (11,3 milhões de habitantes) e Santa Catarina (7 milhões)  apresentam índices mais elevados, indicando que fatores regionais (como acesso a serviços de saúde mental, condições socioeconômicas e culturais) podem influenciar mais fortemente a incidência do que a densidade populacional.

Pergunta 03: Analisar o número de suicídios por gênero

Esta pergunta busca entender a distribuição dos casos de suicídio entre homens e mulheres, visando identificar se há uma prevalência significativa em um dos gêneros


In [0]:
%sql
SELECT sexo, COUNT(*) AS total_obitos
FROM suicidios
where sexo != 'NA'
GROUP BY sexo
ORDER BY total_obitos DESC

sexo,total_obitos
Masculino,43667
Feminino,11803


Conforme o resultado apresentado, a mortalidade por suicídio é 3 vezes maior entre homens comparado às mulheres, padrão consistente com as estatísticas globais de saúde. Segundo a Organização Mundial da Saúde (OMS, 2021), essa disparidade de gênero se mantém em quase todos os países, sendo atribuída a uma combinação de fatores:

Métodos mais letais: Homens tendem a utilizar meios mais violentos e definitivos, como armas de fogo (enquanto mulheres frequentemente optam por intoxicações, que permitem maior chance de intervenção).

Cultura de gênero: A construção social da masculinidade dificulta a busca por ajuda psicológica. Conforme estudo do Instituto de Psiquiatria da USP (2019), 72% dos homens com depressão nunca procuraram tratamento.

Fatores socioeconômicos: Desemprego e pressão para prover o sustento familiar impactam desproporcionalmente os homens, conforme aponta pesquisa da Fiocruz (2020).

Para este resultado foram desconsiderados os valores não apresentados 'NA'.

Pergunta 04: Analisar o número de suicídios por faixa etária

Esta pergunta tem como finalidade mapear as faixas etárias mais afetadas pelo suicídio, permitindo identificar se há grupos de risco.

In [0]:
%sql
SELECT  
CASE   
    WHEN idade BETWEEN 0 AND 9 THEN '0-9'
    WHEN idade BETWEEN 10 AND 19 THEN '10-19'
    WHEN idade BETWEEN 20 AND 29 THEN '20-29'
    WHEN idade BETWEEN 30 AND 39 THEN '30-39'
    WHEN idade BETWEEN 40 AND 49 THEN '40-49'
    WHEN idade BETWEEN 50 AND 59 THEN '50-59'
     ELSE '60+'
    END AS faixa_etaria,
    COUNT(*) AS total_obitos
FROM suicidios  
GROUP BY faixa_etaria
ORDER BY faixa_etaria

faixa_etaria,total_obitos
0-9,11
10-19,3965
20-29,10791
30-39,11710
40-49,10191
50-59,8671
60+,10145


Observamos a maior mortalidade por suicídio na faixa de 30-39 anos (21% dos casos) no período analisado. Conforme pesquisas, três fatores principais ajudam a explicar esse resultado:

Transição de vida crítica: Esta é uma fase marcada por pressões profissionais, familiares e sociais intensas. Conforme estudo da Fiocruz (2021), 68% dos adultos nesta faixa etária relatam níveis críticos de estresse devido a:

Cobranças por estabilidade financeira

Crises em relacionamentos

Dificuldades na progressão de carreira

Acesso a meios letais: Dados do IPEA (2020) mostram que nesta idade há maior posse de armas e acesso a recursos financeiros para obtenção de métodos mais eficazes.

Diagnósticos tardios de saúde mental: Pesquisa da ABP (Associação Brasileira de Psiquiatria, 2022) indica que:

Transtornos como depressão e ansiedade frequentemente se agravam nesta fase

Há resistência em buscar ajuda por "medo de prejudicar a carreira"

O pico de diagnósticos de bipolaridade ocorre entre 30-35 anos

Pergunta 05: Identificar o número de suicídio por estado civil. 

Esta pergunta busca investigar se o estado civil (solteiro, casado, divorciado, viúvo) está associado a uma maior ou menor incidência de suicídios, contribuindo para compreender o impacto das relações interpessoais nesse contexto


In [0]:
%sql
SELECT estciv, COUNT(*) AS total_obitos
FROM suicidios
WHERE estciv IS NOT NULL and estciv != 'NA'
GROUP BY estciv
ORDER BY total_obitos DESC

estciv,total_obitos
Solteiro,27697
Casado,14764
Separado judicialmente,3958
Uniao Consensual,2960
Viuvo,2057


O resultado nos informa que pessoas solteiras representam metade de todos os casos de suicídio no Brasil, mas é importante considerar que esses números podem estar subnotificados, visto que muitas pessoas possuem relacionamentos estáveis sem oficializá-los no cartório, e acabam sendo registradas como 'solteiras', o que pode mascarar a realidade. Alguns fatores podem explicar este resultado encontrado. 

 Falta de rede de apoio: Pessoas casadas geralmente contam com o apoio do cônjuge e da família, enquanto solteiros podem se sentir mais sozinhos em momentos difíceis.

Pressão social: A sociedade muitas vezes cobra que as pessoas formem família, o que pode causar frustração em quem está solteiro.

Problemas econômicos: Solteiros podem ter mais dificuldades financeiras, já que não dividem as contas com um parceiro.

Saúde mental: Estudos mostram que pessoas solteiras tendem a procurar menos ajuda psicológica quando precisam.

Para este resultado foram desconsiderados os dados nulos e não apresentados 'NA'.

Pergunta 06: Analisar o número de suicídios por gênero e raça

Esta pergunta busca cruzar dados de gênero e raça para identificar possíveis interseccionalidades que possam influenciar as taxas de suicídio, permitindo uma análise mais detalhada de grupos específicos que podem estar em maior vulnerabilidade.


In [0]:
%sql
SELECT sexo, racacor, COUNT(*) AS total_obitos
FROM 
suicidios
WHERE 
sexo IS NOT NULL AND racacor IS NOT NULL 
GROUP BY 
sexo, 
racacor
ORDER BY 
total_obitos DESC

sexo,racacor,total_obitos
Masculino,Branca,21362
Masculino,Parda,18395
Feminino,Branca,6505
Feminino,Parda,4309
Masculino,Preta,2341
Feminino,Preta,503
Masculino,Indigena,452
Masculino,Amarela,168
Feminino,Indigena,159
Feminino,Amarela,54


Os homens brancos e pardos lideram estatísticas de suicídio no período analisado, reforçando a vulnerabilidade deste público, que conforme pesquisas, estão relacionadas a fatores como pressões socioculturais específicas como: Expectativa de provedor, acesso a métodos letais e barreiras (baixa procura) no cuidado em saúde mental.

Para este resultado foram desconsiderados os dados nulos.

Pergunta 07: Identificar se o grau de escolaridade interfere na taxa de suicídio.

Esta pergunta busca avaliar se há uma correlação entre o nível de escolaridade e as taxas de suicídio, permitindo entender se fatores educacionais podem influenciar o comportamento suicida.



In [0]:
%sql
SELECT esc, COUNT(*) AS total_obitos
FROM suicidios
WHERE esc IS NOT NULL and esc != 'NA' 
GROUP BY esc
ORDER BY total_obitos DESC

esc,total_obitos
8 a 11 anos,13758
4 a 7 anos,13638
1 a 3 anos,7566
12 e mais,5240
Nenhuma,2469


Este resultado nos demonstrou que, diferente do esperado, pessoas com escolaridade média (8 a 11 anos) e baixa (4 a 7 anos) representam o público com o maior índice de óbitos por suicídio no Brasil. Essa realidade pode ser compreendida por meio de três dimensões principais:

Indivíduos com ensino superior completo têm 40% menos risco (OMS, 2022)

Pessoas com até 3 anos de estudo frequentemente vivem em comunidades com forte apoio comunitário

Já quem possui 4 a 11 anos de estudo enfrenta o pior cenário: qualificação insuficiente para boas oportunidades, mas suficiente para gerar frustração.